Here is a sample code for gru4rec using recbole. 
I made it based on the following code from a past competition
https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

I think you can get a better score if you change the training data or epoch size, etc. Enjoy!

In [ ]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 50.4 MB/s eta 0:00:00


In [ ]:
!pip install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.7 MB/s eta 0:00:00


In [ ]:
import tqdm
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import random
import os 
import h5py
import sys
import gc

from matplotlib import pyplot as plt
import pyarrow.parquet as pq

# 1. Create atomic file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#train = pl.read_parquet('/content/QB-video.csv')
#test = pl.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/test.parquet')
df = pd.read_csv('/content/drive/MyDrive/sbr_data_1M.csv', nrows=1000000)
#df = pd.read_csv('/content/QB-video.csv')
#df = pl.read_parquet('../input/otto-train-and-test-data-for-local-validation/test.parquet')

#df = df.sort_values(['user_id', 'item_id', 'watching_times', 'gender'])
#df = df.with_columns((pl.col('watching_times') * 1e9).alias('watching_times'))
#df = df.rename({'user_id': 'user_id:token', 'item_id': 'item_id:token','watching_times': 'watching_times:float'})

mapping = {df.columns[0]:'user_id:token', df.columns[1]: 'item_id:token', df.columns[7]:'watching_times:float'}
df = df.rename(columns=mapping)

df = df.drop(columns=['click', 'follow', 'like', 'share', 'video_category', 'age'])
#df = df.sample(frac =.10)
df1 = df.loc[df['gender'] == 0]
df2 = df.loc[df['gender'] == 1]
df3 = df.loc[df['gender'] == 2]
df4 = df.drop(columns=['gender'])
df1 = df1.drop(columns=['gender'])
df2 = df2.drop(columns=['gender'])
df3 = df3.drop(columns=['gender'])

In [ ]:
print(df1.shape[0])
print(df2.shape[0])
print(df3.shape[0])
print(df4.shape[0])

196165
605173
198662
1000000


In [ ]:
#!mkdir /kaggle/working/recbox_data
#df['user_id:int64', 'item_id:int64', 'watching_times:int64'].write_csv('/content/data.inter', sep='\t')

#del df, train, test
#gc.collect()

df3.to_csv('/content/data/data.inter', index=False, sep='\t')

In [ ]:
print(df.head())
print(df.dtypes)

   user_id:token  item_id:token  watching_times:float  gender
0              1              1                     1       1
1              1              2                     1       1
2              1              3                     1       1
3              1          80936                     1       1
4              1            781                     1       1
user_id:token           int64
item_id:token           int64
watching_times:float    int64
gender                  int64
dtype: object


# 3. Create dataset and train model with Recbole

For anyone need instruction document, please check this link: https://recbole.io/docs/user_guide/usage/use_modules.html

In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import BERT4Rec, GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

from recbole.utils.case_study import full_sort_topk

In [ ]:
MAX_ITEM = 20

parameter_dict = {
    'data_path': '/content/',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'watching_times',
    'user_inter_num_interval': "[5,Inf)",
    'item_inter_num_interval': "[5,Inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'watching_times'], 'item': ['item_id', 'item_emb']},
    'train_neg_sample_args': None,
    'epochs': 1,
    'stopping_step':3,
    
    'eval_batch_size': 1024,
    #'train_batch_size': 1024,
#    'enable_amp':True,
    'MAX_ITEM_LIST_LENGTH': MAX_ITEM,
    'eval_args': {
        'split': {'RS': [9, 1, 0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}

config = Config(model='BERT4Rec', dataset='data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
# model loading and initialization
model = BERT4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
#best_valid_score, best_valid_result = trainer.fit(train_data)

In [ ]:
print(best_valid_score)
print(best_valid_result)
import gc
del trainer, train_data, valid_data, test_data

0.0106
OrderedDict([('recall@10', 0.0332), ('mrr@10', 0.0106), ('ndcg@10', 0.0159), ('hit@10', 0.0332), ('precision@10', 0.0033)])


In [ ]:
gc.collect()

0